In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
seed = 2
from xgboost import XGBClassifier
import pickle

In [2]:
#train = pd.read_csv('./data/new_train.csv')
train = pd.read_csv('./data/train_sample.csv')

In [3]:
sample_size = int(1e4)
try :
    sample = train[train.target == 1].sample(n=sample_size/2, random_state=seed)
    sample = sample.append( train[train.target == 0].sample(n=sample_size/2, random_state=seed))
except ValueError :
    sample = train[train.target == 1]
    sample = sample.append( train[train.target == 0].sample(n=sample_size-21694, random_state=seed) )

sample = sample.sample(frac=1, random_state=seed).reset_index(drop=True)
#sample.to_csv('./data/train_sample.csv', index=False)
train = sample

In [4]:
target = train.target
train.drop(['target','id'], inplace=True, axis=1)
train.drop([ col for col in train.columns if col.startswith('ps_cont') ],axis=1, inplace=True)

In [5]:
with open('./data/OneHotEncoder.clf', 'rb') as f:
    encoders = pickle.load(f)

In [6]:
enc_train = None

for feature,encoder in zip(train.columns,encoders) :
    encoded = encoder.transform(train[feature].values.reshape(-1,1))
    if enc_train is None :
        enc_train = encoded
    else :
        enc_train = np.concatenate((enc_train, encoded), axis=1)

In [ ]:
train = sample
#del train
#del enc_train

In [ ]:
with open('./data/model.pkl','rb') as f :
    model = pickle.load(f)

In [13]:
pred = model.predict(x_test)

In [15]:
from sklearn.metrics import classification_report
print classification_report(y_true=y_test, y_pred=pred)

             precision    recall  f1-score   support

          0       0.57      0.63      0.60       511
          1       0.57      0.51      0.54       489

avg / total       0.57      0.57      0.57      1000



In [10]:
def split_data(X, Y, ratio=0.1):
    test_ids = np.random.randint(0, X.shape[0], int(X.shape[0] * ratio))
    x_test = X[test_ids]
    y_test = Y[test_ids]
    x_train = np.delete(X, test_ids, axis=0)
    y_train = Y.drop(test_ids)
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = split_data(enc_train, target, .1)

In [12]:
model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, eval_metric='error', gamma=0,
       learning_rate=0.2, max_delta_step=0, max_depth=1,
       min_child_weight=7, missing=None, n_estimators=100, n_jobs=4,
       nthread=None, objective='binary:logistic', random_state=2,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.4)

In [9]:
model = XGBClassifier(
	objective = 'binary:logistic',
	n_jobs = 4,
	random_state = seed,
	eval_metric = "error",
	#-------
	max_depth = 1,
	min_child_weight = 7,
	#----------
	gamma = 0,
	#----------
	subsample = 0.4,
	colsample_bytree = 0.4,
	#------------
	scale_pos_weight = 1,
	#-----------
	reg_alpha = 0,
	reg_lambda = 1,
	#------------
	learning_rate = 0.2

	#n_estimators = 1000,
	)

In [ ]:
with open('./data/model.pkl', 'wb') as f:
    pickle.dump(file=f, obj=model)

In [2]:
params = {
    
    #-----------------------------------------------------------------------
    # dealing with imblanced data
    #-------------------------------------------------------------------------------
    'max_delta_step':0,
    #Maximum delta step we allow each tree's weight estimation to be. If the value is set to 0,
    #it means there is no constraint. If it is set to a positive value, it can help making the update step more 
    #conservative. Usually this parameter is not needed, but it might help in logistic regression when class is
    #extremely imbalanced. Set it to value of 1-10 might help control the update.
    #default:0, range[0-inf]
    'scale_pos_weight' : 1,  # control balance between +ve and -ve weights, default:1
    
    #--------------------------------------------------------------------------------------
    #  Regularization
    #-------------------------------------------------------------------------------
    'alpha' : 0,
    #L1 regularization term on weights, increase this value will make model more conservative, default:0
    'lambda' : 1,
    #L2 regularization term on weights, increase this value will make model more conservative, default1.
    
    #------------------------------------------------------
    # add randomness to make training robust to noise
    #----------------------------------------------------------------
    'subsample' : 1,
    #ratio of the training instance. Setting it to 0.5 means that XGBoost randomly collected half of the data 
    #instances to grow trees and this will prevent overfitting. default:1, range[0-1]
    'colsample_bytree' : 1, #subsample ratio of columns when constructing each tree.default:1, range[0-1]
    'colsample_bylevel' : 1, #subsample ratio of columns for each split, in each level. default:1, range[0-1]
    
    #-------------------------------------------------------------
    # Tree parameters
    #--------------------------------------------------
    'min_child_weight': 1, # minimum sum of instance weight (hessian) needed in a child. 
    # the more conservative the algorithm will be. default:1, range[0-inf]
    'max_depth' : 6,
    # maximum depth of a tree, increase this value will make the model more complex / likely to be overfitting
    # default : 6, range[0-inf]
    
    #-----------------------------------------------------------------------
    # LOSS reduction
    #------------------------------------------
    'gamma':0, #minimum loss reduction required to make a further partition on a leaf node of the tree.
    #The larger, the more conservative the algorithm will be. default : 0, range[0-inf]
    'eta':0.3, #step size shrinkage used in update to prevents overfitting. default:0.3, range[0-1]
    
    #---------------------------------------------------------------------------------------------------
    #Learning Task Parameters
    #------------------------------------------------------------------------------
    'objective': 'reg:linear', # reg:logistic, binary:logistic, multi:softmax
    'eval_metric' : 'error', # error for binary class., merror for multiclass classification.
    'seed' : 0
}

SyntaxError: invalid syntax (<ipython-input-2-b8f2eead6bb3>, line 30)